In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/homecreditprocessed/train_application_bureau_with_label.csv
/kaggle/input/homecreditprocessed/test_application_bureau_drop_0importance.csv
/kaggle/input/homecreditprocessed/test_bureau.csv
/kaggle/input/homecreditprocessed/train_bureau2.csv
/kaggle/input/homecreditprocessed/train_application_bureau.csv
/kaggle/input/homecreditprocessed/test_application_previous_without_collinearity.csv
/kaggle/input/homecreditprocessed/train_application_bureau_drop_0importance.csv
/kaggle/input/homecreditprocessed/train_application_previous_dropped_collinearity.csv


In [3]:
Train = pd.read_csv("/kaggle/input/homecreditprocessed/train_application_bureau_with_label.csv")
train_df = Train.drop(['SK_ID_CURR','TARGET'], axis=1)
X = train_df
Y = Train["TARGET"]
Test = pd.read_csv("/kaggle/input/homecreditprocessed/test_bureau.csv")
test_df = Test.drop(['SK_ID_CURR'], axis=1)
X.head()


NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0         Cash loans           M            N               Y             0   
1         Cash loans           F            N               N             0   
2    Revolving loans           M            Y               Y             0   
3         Cash loans           F            N               Y             0   
4         Cash loans           M            N               Y             0   

   AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE NAME_TYPE_SUITE  \
0          202500.0    406597.5      24700.5         351000.0   Unaccompanied   
1          270000.0   1293502.5      35698.5        1129500.0          Family   
2           67500.0    135000.0       6750.0         135000.0   Unaccompanied   
3          135000.0    312682.5      29686.5         297000.0   Unaccompanied   
4          121500.0    513000.0      21865.5         513000.0   Unaccompanied   

   ... client_bureau_balance_STATUS_X_count_max  \
0  ...                                      3.0   
1  ...                                      NaN   
2  ...                                      NaN   
3  ...                                      NaN   
4  ...                                      NaN   

  client_bureau_balance_STATUS_0_count_max  \
0                                     18.0   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

  client_bureau_balance_STATUS_C_count_mean  \
0                                     2.875   
1                                       NaN   
2                                       NaN   
3                                       NaN   
4                                       NaN   

  client_bureau_balance_MONTHS_BALANCE_count_mean  \
0                                           13.75   
1                                             NaN   
2                                             NaN   
3                                             NaN   
4                                             NaN   

   client_bureau_balance_STATUS_X_count_sum  \
0                                      15.0   
1                                       0.0   
2                                       0.0   
3                                       NaN   
4                                       0.0   

   client_bureau_balance_STATUS_0_count_sum  \
0                                      45.0   
1                                       0.0   
2                                       0.0   
3                                       NaN   
4                                       0.0   

   client_bureau_balance_STATUS_C_count_max  \
0                                      13.0   
1                                       NaN   
2                                       NaN   
3                                       NaN   
4                                       NaN   

   client_bureau_balance_MONTHS_BALANCE_count_max  \
0                                            22.0   
1                                             NaN   
2                                             NaN   
3                                             NaN   
4                                             NaN   

   client_bureau_balance_STATUS_C_count_sum  \
0                                      23.0   
1                                       0.0   
2                                       0.0   
3                                       NaN   
4                                       0.0   

   client_bureau_balance_MONTHS_BALANCE_count_sum  
0                                           110.0  
1                                             0.0  
2                                             0.0  
3                                             NaN  
4                                             0.0  

[5 rows x 307 columns]

In [4]:
train_df=pd.get_dummies(train_df)
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'median') # Median imputation of missing values
scaler = MinMaxScaler(feature_range = (0, 1)) # Scale each feature to 0-1
for column in train_df.columns:
    train_df[[column]] = imputer.fit_transform(train_df[[column]])
    train_df[[column]] = scaler.fit_transform(train_df[[column]])
X = train_df
Y = Train["TARGET"]
X.head()

CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE  \
0           0.0          0.001512    0.090287     0.090032         0.077441   
1           0.0          0.002089    0.311736     0.132924         0.271605   
2           0.0          0.000358    0.022472     0.020025         0.023569   
3           0.0          0.000935    0.066837     0.109477         0.063973   
4           0.0          0.000819    0.116854     0.078975         0.117845   

   REGION_POPULATION_RELATIVE  DAYS_BIRTH  DAYS_EMPLOYED  DAYS_REGISTRATION  \
0                    0.256321    0.888839       0.045086           0.852140   
1                    0.045016    0.477114       0.043648           0.951929   
2                    0.134897    0.348534       0.046161           0.827335   
3                    0.107023    0.350846       0.038817           0.601451   
4                    0.392880    0.298591       0.038820           0.825268   

   DAYS_ID_PUBLISH  ...  HOUSETYPE_MODE_terraced house  \
0         0.705433  ...                            0.0   
1         0.959566  ...                            0.0   
2         0.648326  ...                            0.0   
3         0.661387  ...                            0.0   
4         0.519522  ...                            0.0   

   WALLSMATERIAL_MODE_Block  WALLSMATERIAL_MODE_Mixed  \
0                       0.0                       0.0   
1                       1.0                       0.0   
2                       0.0                       0.0   
3                       0.0                       0.0   
4                       0.0                       0.0   

   WALLSMATERIAL_MODE_Monolithic  WALLSMATERIAL_MODE_Others  \
0                            0.0                        0.0   
1                            0.0                        0.0   
2                            0.0                        0.0   
3                            0.0                        0.0   
4                            0.0                        0.0   

   WALLSMATERIAL_MODE_Panel  WALLSMATERIAL_MODE_Stone, brick  \
0                       0.0                              1.0   
1                       0.0                              0.0   
2                       0.0                              0.0   
3                       0.0                              0.0   
4                       0.0                              0.0   

   WALLSMATERIAL_MODE_Wooden  EMERGENCYSTATE_MODE_No  EMERGENCYSTATE_MODE_Yes  
0                        0.0                     1.0                      0.0  
1                        0.0                     1.0                      0.0  
2                        0.0                     0.0                      0.0  
3                        0.0                     0.0                      0.0  
4                        0.0                     0.0                      0.0  

[5 rows x 431 columns]

In [5]:
test_df=pd.get_dummies(test_df)
for column in test_df.columns:
    test_df[[column]] = imputer.fit_transform(test_df[[column]])
    test_df[[column]] = scaler.fit_transform(test_df[[column]])
test_df

CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
0              0.00          0.024654    0.238037     0.102453   
1              0.00          0.016440    0.080785     0.084558   
2              0.00          0.040054    0.280965     0.378515   
3              0.10          0.065721    0.695297     0.262078   
4              0.05          0.034920    0.263804     0.166995   
...             ...               ...         ...          ...   
48739          0.00          0.021574    0.167035     0.085138   
48740          0.10          0.029787    0.262401     0.166111   
48741          0.05          0.040054    0.122699     0.173381   
48742          0.00          0.045187    0.184049     0.128073   
48743          0.00          0.024654    0.121685     0.125726   

       AMT_GOODS_PRICE  REGION_POPULATION_RELATIVE  DAYS_BIRTH  DAYS_EMPLOYED  \
0             0.184049                    0.257380    0.333427       0.039545   
1             0.061350                    0.491855    0.399339       0.033953   
2             0.265849                    0.260854    0.288794       0.033982   
3             0.695297                    0.361760    0.628269       0.040755   
4             0.263804                    0.135340    0.680685       0.039905   
...                ...                         ...         ...            ...   
48739         0.102249                    0.024760    0.292602       0.032124   
48740         0.204499                    0.491855    0.784510       0.042628   
48741         0.122699                    0.361760    0.519292       0.037695   
48742         0.184049                    0.257380    0.628717       0.038494   
48743         0.102249                    0.088243    0.629053       0.043976   

       DAYS_REGISTRATION  DAYS_ID_PUBLISH  ...  HOUSETYPE_MODE_terraced house  \
0               0.782059         0.872086  ...                            0.0   
1               0.615631         0.744329  ...                            0.0   
2               0.908313         0.448173  ...                            0.0   
3               0.915690         0.337114  ...                            0.0   
4               0.831380         0.328607  ...                            0.0   
...                  ...              ...  ...                            ...   
48739           0.616643         0.464556  ...                            0.0   
48740           0.872903         0.526938  ...                            0.0   
48741           0.886983         0.763075  ...                            0.0   
48742           0.938412         0.785129  ...                            0.0   
48743           0.954810         0.335224  ...                            0.0   

       WALLSMATERIAL_MODE_Block  WALLSMATERIAL_MODE_Mixed  \
0                           0.0                       0.0   
1                           0.0                       0.0   
2                           0.0                       0.0   
3                           0.0                       0.0   
4                           0.0                       0.0   
...                         ...                       ...   
48739                       0.0                       0.0   
48740                       0.0                       0.0   
48741                       0.0                       0.0   
48742                       0.0                       0.0   
48743                       0.0                       0.0   

       WALLSMATERIAL_MODE_Monolithic  WALLSMATERIAL_MODE_Others  \
0                                0.0                        0.0   
1                                0.0                        0.0   
2                                0.0                        0.0   
3                                0.0                        0.0   
4                                0.0                        0.0   
...                              ...                        ...   
48739                            0.0                        0.0   

In [6]:
train_df, test_df = train_df.align(test_df, join = 'inner', axis = 1)
train_df['training_set'] = True 
test_df['training_set'] = False
df_full = pd.concat([train_df, test_df])
train_df = df_full[df_full['training_set']==True]
train_df = train_df.drop('training_set', axis=1)
test_df = df_full[df_full['training_set']==False]
test_df = test_df.drop('training_set', axis=1)

In [7]:
print('Size of training_set: ', train_df.shape)
print('Size of testing_set: ', test_df.shape)

Size of training_set:  (307511, 428)
Size of testing_set:  (48744, 428)


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(train_df, Y)
predictions = model.predict(test_df)


output = pd.DataFrame({'SK_ID_CURR': Test['SK_ID_CURR'], 'TARGET': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")


ValueError: Number of features of the model must match the input. Model n_features is 431 and input n_features is 428 